<a href="https://colab.research.google.com/github/syrma/RLExp/blob/master/PPO_VECENV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install tensorflow
!pip install pybullet
!pip install gym_vecenv
!pip install tensorflow_probability

import tensorflow as tf
import gym
import pybullet_envs
import time
import math
import gym_vecenv
import tensorflow_probability as tfp
tfd = tfp.distributions


In [78]:
@tf.function
def action(model, obs, env):
    est = model(obs)
    if env.action_space.shape:
        dist = tfd.MultivariateNormalDiag(est, tf.exp(model.log_std))
    else:
        dist = tfd.Categorical(logits=est, dtype=env.action_space.dtype)

    action = dist.sample()
    logprob = tf.reduce_sum(dist.log_prob(action))

    return action, logprob

In [79]:
    size = 5000
    epochs = 100
    opt = tf.optimizers.Adam(learning_rate=1e-2)
    γ = .99
    λ = 0.97
    num_env = 5

    env_name = "CartPole-v0"

    env = gym_vecenv.DummyVecEnv([lambda: gym.make(env_name)] * num_env)
    # policy/actor model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='tanh', input_shape=env.observation_space.shape),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(env.action_space.shape[0] if env.action_space.shape else env.action_space.n)
    ])
    if env.action_space.shape:
        model.log_std = tf.Variable(tf.fill(env.action_space.shape, -0.5))
    model.summary()

    # value/critic model
    value_model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='tanh', input_shape=env.observation_space.shape),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    value_model.compile('adam', loss='MSE')
    value_model.summary()


Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 64)                320       
_________________________________________________________________
dense_55 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_56 (Dense)             (None, 2)                 130       
Total params: 4,610
Trainable params: 4,610
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 64)                320       
_________________________________________________________________
dense_58 (Dense)             (None, 64)                4160      
____________________________

In [80]:
    obs_spc = env.observation_space
    act_spc = env.action_space
    start_time = time.time()
    
    obs_buf = tf.TensorArray(obs_spc.dtype, size)
    act_buf = tf.TensorArray(act_spc.dtype, size)
    rew_buf = tf.TensorArray(tf.float32, size)
    prob_buf = tf.TensorArray(tf.float32, size)
    done_buf = tf.TensorArray(tf.float32, size)
    
    obs = env.reset()
    obs = tf.cast(obs, obs_spc.dtype)

    for i in range(size):
        act, prob = action(model, obs, env)
        new_obs, rew, done, _ = env.step(act.numpy())

        obs_buf = obs_buf.write(i, obs)
        act_buf = act_buf.write(i, act)
        rew_buf = rew_buf.write(i, rew)
        prob_buf = prob_buf.write(i, prob)
        done_buf = done_buf.write(i, done)

        obs = tf.cast(new_obs, obs_spc.dtype)

    obs_buf = obs_buf.stack()
    act_buf = act_buf.stack()
    rew_buf = rew_buf.stack()
    prob_buf = prob_buf.stack()
    done_buf = done_buf.stack()

    # last_val is 0 when done
    last_val = tf.squeeze(value_model(obs)) * (1 - done_buf[-1])

    #rets = []
    #lens = []


In [81]:
#TODO: turn into a list of tensor arrays
v_hats = [tf.TensorArray(tf.float32, size) for _ in range(num_env)]
gae = [tf.TensorArray(tf.float32, size) for _ in range(num_env)]

#TODO: changer la boucle et remplacer cumprod/cumsum
last_idx = [0] * num_env

for i in range(size):
  for j in range(num_env): # num_env = ?
    if i != size - 1 and not done_buf[i,j]:
      continue

    # sum of discounted rewards
    current_episode = slice(last_idx[j],i+1)   
    ep_idx = range(last_idx[j], i+1) 
    ep_rew = rew_buf[current_episode, j]
    discounts = tf.math.cumprod(tf.fill(ep_rew.shape, γ), exclusive=True)
    ep_v_hats = tf.math.cumsum(discounts * ep_rew, reverse=True)
    v_hats[j] = v_hats[j].scatter(ep_idx, ep_v_hats)

    Vs = tf.squeeze(value_model(obs_buf[current_episode, j]), axis=1)
    if i == size - 1:
      Vsp1 = tf.concat([Vs[1:], [last_val[j]]], axis=0)
    else:
      Vsp1 = tf.concat([Vs[1:], [0]], axis=0)
    
    deltas = rew_buf[current_episode, j] + γ * Vsp1 - Vs

    # compute the advantage function (gae)
    discounts = tf.math.cumprod(tf.fill(deltas.shape, γ * λ), exclusive=True)
    ep_gae = tf.math.cumsum(discounts * deltas, reverse=True)
    gae[j] = gae[j].scatter(ep_idx, ep_gae)

    last_idx[j] = i+1

v_hats = [v_hat.stack() for v_hat in v_hats]
gae = [g.stack() for g in gae]

In [82]:
#def run_env(env, size, model, value_model, γ, λ):

In [83]:
def loss():
    eps = 0.1
    obs, act, adv, logprob = obs_buf, act_buf, gae, prob_buf

    if act_spc.shape:
        dist = tfd.MultivariateNormalDiag(model(obs), tf.exp(model.log_std))
    else:
        dist = tfd.Categorical(logits=model(obs))
    new_logprob = dist.log_prob(act)

    #use map?
    #map(

    mask = map(adv, tf.float32, tf.cast)

    masks = [tf.cast(adv[_] >= 0, tf.float32) for _ in range(num_env)]
    epsilon_clip = masks * (1 + eps) + (1 - masks) * (1 - eps)
    ratio = tf.exp(new_logprob - logprob)

    return -tf.reduce_mean(tf.minimum(ratio * adv, epsilon_clip * adv))

In [84]:
# train one epoch

train_start_time = time.time()

var_list = list(model.trainable_weights)
if act_spc.shape:
    var_list.append(model.log_std)

opt.minimize(loss(), var_list=var_list)

train_time = time.time() - train_start_time
run_time = train_start_time - start_time

print('run time', run_time, 'train time', train_time)
print('AvgEpRet:', tf.reduce_mean(rets).numpy())

hist = value_model.fit(obs_buf.numpy(), V_hats.numpy(), batch_size=32)

TypeError: 'DType' object is not iterable